## AVISO:
Este notebook es provisional, la versión final estará en training.py

In [1]:
# Para 'intents.json'
import json # Para el formato json
import codecs # Lectura de caracteres en español

# Procesamiento de Lenguaje Natural
import spacy # Lemmatizer (convertir palabras) con lenguaje español
import pickle
import numpy as np
import nltk # Natural Language ToolKit: Tokenizar

# Redes Neuronales:
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
nlp = spacy.load('es_core_news_sm')

def lemmatizer(text):  
    doc = nlp(text)
    return ' '.join([word.lemma_ for word in doc])

text = '¿personas?, ¿estás ahí?'
print(lemmatizer(text))

¿ persona ? , ¿ estar ahí ?


In [3]:
intents = json.loads(codecs.open('intents.json', encoding='utf-8').read())

In [14]:
words = [] # Palabras individuales usadas
tags = [] # Etiquetas de intención
ignore_letters = ['¿', '?', '.', '!', '(', ')']
documents = [] # Lista de tuplas ([Lista de palabras], etiqueta asociada de intención)
for intent in intents['intents']:
    # Por cada patrón:
    for pattern in intent['patterns']:
        #pattern = lemmatizer(pattern)
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

words = [lemmatizer(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
tags = sorted(set(tags))
print("Words: %s \n"%words)
print("Tags: %s \n"%tags)
print("Documents:", documents)

Words: ['Buenas', 'Buenos', 'Hey', 'Hi', 'Hola', 'aqui', 'aquí', 'donde', 'día', 'dónde', 'estar', 'este', 'llegar', 'llevar', 'llévame', 'lugar', 'noche', 'poder', 'quedar', 'tarde', '¿ cómo', '¿ dónde', '¿ saber', '¿ yo'] 

Tags: ['Consulta_Lugar', 'Saludos'] 

Documents: [(['Hola'], 'Saludos'), (['Hey'], 'Saludos'), (['Hi'], 'Saludos'), (['Buenos', 'días'], 'Saludos'), (['Buenas', 'tardes'], 'Saludos'), (['Buenas', 'noches'], 'Saludos'), (['Buenas'], 'Saludos'), (['¿Sabes', 'donde', 'está', '?'], 'Consulta_Lugar'), (['¿Dónde', 'queda', '?'], 'Consulta_Lugar'), (['¿Dónde', 'queda', 'este', 'lugar', '?'], 'Consulta_Lugar'), (['¿Cómo', 'puedo', 'llegar', '?'], 'Consulta_Lugar'), (['¿sabes', 'dónde', 'queda', '?'], 'Consulta_Lugar'), (['¿me', 'puedes', 'llevar', 'aqui', '?'], 'Consulta_Lugar'), (['llévame', 'aquí'], 'Consulta_Lugar')]


In [16]:
training = []
output_empty = [0] * len(tags)

for document in documents:
    bag = [] # bag of words
    word_patterns = document[0]
    word_patterns = [lemmatizer(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word.lower() in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[tags.index(document[1])] = 1
    training.append([bag, output_row])

print(training)

[[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1]], [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1]], [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1]], [[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0], [1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 